# Todo?

Handle extreme data

remove meanwdspd outliers?

In [ ]:
# Call describe on df and transpose it due to the large number of columns
spread = df.describe().T

# precalculate interquartile range for ease of use in next calculation
IQR = spread['75%'] - spread['25%']

# create an outliers column which is either 3 IQRs below the first quartile or
# 3 IQRs above the third quartile
spread['outliers'] = (spread['min']<(spread['25%']-(3*IQR)))|(spread['max'] > (spread['75%']+3*IQR))

# just display the features containing extreme outliers
spread[spread.outliers]
# spread

lasso regression and ridge regression to handle too many features?

lassso: seletive on features

ridge: limit features

$\uparrow$ multiple models and compare?

Similarity-based learning and predict weather type?

need to re-do dataPreparation to include 'special' or 'important'